# Классификация диалектизмов архангельских говоров

В этом ноутбуке представлена модель для классификации диалектизмов, обученная на материале архангельских говоров.

Продемонстрируем, как использовать эту модель для классификации новых слов.


In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

In [6]:
# в словаре используются следующие символы
vocab = 'абвгдежзийклмнопрстуфхцчшщъыьэюя'
vocab_to_int = {word: i + 1 for i, word in enumerate(vocab)}

# паддинговый символ
vocab_to_int['PAD'] = 0

vocab_size = len(vocab_to_int.keys())

# метки классов
dec = ['Фонетическая',
       'Грамматическая',
       'Словообразовательная',
       'Лексическая']


def encode(text, max_char_length=20):
    '''
    Преобразует текст в формат One-Hot Encoding (OHE).

    Параметры:
    text (str): Входной текст, который нужно преобразовать.
    max_char_length (int): Максимальная длина текста для преобразования.
                           По умолчанию 20.

    Возвращает:
    np.ndarray: Массив размером [max_char_length, vocab_size],
                где каждый символ текста преобразован в OHE-вектор.
    '''

    encoded = np.zeros([max_char_length, vocab_size], dtype=int)
    i = 0
    for letter in text:
        if i >= max_char_length:
            break
        if letter.lower() in vocab_to_int:
            encoded[i][vocab_to_int[letter.lower()]] = 1
        i += 1
    return encoded

In [3]:
def encode_word(word, max_char_length=20):
    '''
    Преобразует слово для подачи на вход нейронной сети.

    Параметры:
    word (str): Входное слово, которое нужно преобразовать.
    max_char_length (int): Максимальная длина слова для преобразования.
                           По умолчанию 20.

    Возвращает:
    np.ndarray: Массив размером [max_char_length, vocab_size],
                где каждый символ текста преобразован в OHE-вектор.
    '''

    word = encode(word.lower().replace('ё', 'е'))
    word = word.astype('float32')
    word = word.reshape(1, max_char_length, vocab_size)
    return word

Для примера классификации использованы следующие слова:

*   "наблюдник" - словообразовательная категория,
*   "рукотерник" - словообразовательная категория,
*   "жырнов" - фонетическая категория,
*   "улец" - фонетическая категория,
*   "складь" - грамматическая категория,
*   "бывата" - грамматическая категория,
*   "катанцы" - лексическая категория,
*   "халупа" - лексическая категория.





In [7]:
words = ['наблюдник',
         'рукотерник',
         'жырнов',
         'улец',
         'складь',
         'бывата',
         'катанцы',
         'халупа']
test_data = {w: encode_word(w) for w in words}

In [8]:
model = load_model('cnn_uni.keras')
for word, encoded_word in test_data.items():
    prediction = model.predict(encoded_word)
    print(f'{word} принадлежит к категории: ' \
          f'{np.argmax(prediction), dec[np.argmax(prediction)]}.')
    print(f'Вероятность по всем классам: {prediction}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
наблюдник принадлежит к категории: (2, 'Словообразовательная').
Вероятность по всем классам: [[0.00599452 0.00456156 0.8984673  0.09097663]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
рукотерник принадлежит к категории: (2, 'Словообразовательная').
Вероятность по всем классам: [[0.00917311 0.00525502 0.93487924 0.05069259]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
жырнов принадлежит к категории: (3, 'Лексическая').
Вероятность по всем классам: [[0.13485074 0.11455166 0.21601236 0.53458524]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
улец принадлежит к категории: (0, 'Фонетическая').
Вероятность по всем классам: [[0.5758248  0.14848776 0.08798897 0.18769848]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
складь принадлежит к категории: (3, 'Лексическая').
Вероятность по всем классам: [[0.23075326 0.17887256 0.2208019  0.36957222]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
бывата принадлежит к категории: (0, 'Фонетическая').
Вероятность по всем классам: [[0.3741926  0.12540